In [1]:
from Player import Player
from Trainer import Trainer
from TetrisModel import TetrisModel
from Pretrainer import Pretrainer
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import glob
import time

In [2]:
piece_dim = 8
key_dim = 12
depth = 16
gamma = 0.99
lam = 0.95

In [5]:
# Use lambda instead of gamma to immitate shape of gae without value predictions
pretrainer = Pretrainer(gamma=lam)

In [6]:
players_data = pretrainer._load_data()

In [7]:
pretrainer._load_dset(players_data)

1.00

In [8]:
max_len = pretrainer._max_len

In [16]:
gt_dset = pretrainer._cache_dset()

Done Caching


In [10]:
agent = TetrisModel(piece_dim=piece_dim,
                    key_dim=key_dim,
                    depth=depth,
                    num_heads=4,
                    num_layers=4,
                    max_length=max_len)

In [11]:
logits, values = agent((tf.random.uniform((1, 28, 10, 1)),
                        tf.random.uniform((1, 7), minval=0, maxval=8, dtype=tf.int32),
                        tf.random.uniform((1, max_len-1), minval=0, maxval=key_dim, dtype=tf.int32)))
agent.summary(), tf.shape(logits), tf.shape(values)

Model: "tetris_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (1, 70, 16)               4800      
                                                                 
 seq_embedding (SeqEmbedding  multiple                 128       
 )                                                               
                                                                 
 seq_embedding_1 (SeqEmbeddi  multiple                 192       
 ng)                                                             
                                                                 
 pdec_0 (DecoderLayer)       multiple                  9776      
                                                                 
 pdec_1 (DecoderLayer)       multiple                  9776      
                                                                 
 pdec_2 (DecoderLayer)       multiple                 

(None,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1, 21, 12])>,
 <tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1, 21,  1])>)

In [12]:
agent_optimizer = keras.optimizers.Adam()
agent.compile(optimizer=agent_optimizer)

In [13]:
epochs = 10

In [14]:
agent.load_weights('agent_weights/agent')

In [17]:
losses, accs = pretrainer.train(agent, gt_dset, epochs)


6050	|	Loss: 2.36	|	Accuracy: 0.76	|	Step Time:  55ms	|	
50	|	Loss: 1.41	|	Accuracy: 0.77	|	Step Time:  74ms	|	


KeyboardInterrupt



In [ ]:
plt.plot(losses)
plt.plot(accs)

In [18]:
if 'y' in input('YOU SURE?????'):
    agent.save_weights('agent_weights/agent')

YOU SURE????? y


In [19]:
%matplotlib qt

In [20]:
trainer = Trainer(model=agent,
                  optimizers=(None, None),
                  seq_len=max_len,
                  gamma=gamma,
                  lam=lam,
                  max_episode_steps=500,
                  buffer_cap=10000)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: michaelsherrick. Use `wandb login --relogin` to force relogin


In [24]:
trainer.fill_replay_buffer()

Done filling replay buffer


In [ ]:
trainer.train(gens=1000, train_steps=10, training_actor=True)

Current Gen: 1	|	Avg Reward: 0.0	|	Total Reward: 19.2	|
Current Gen: 2	|	Avg Reward: 0.0	|	Total Reward: 21.9	|
Current Gen: 3	|	Avg Reward: 0.0	|	Total Reward: 14.2	|
Current Gen: 4	|	Avg Reward: 0.0	|	Total Reward: 4.0	|
Current Gen: 5	|	Avg Reward: 0.0	|	Total Reward: 4.8	|
Current Gen: 6	|	Avg Reward: 0.0	|	Total Reward: 36.8	|
Current Gen: 7	|	Avg Reward: 0.0	|	Total Reward: 12.0	|
Current Gen: 8	|	Avg Reward: 0.0	|	Total Reward: 19.8	|
Current Gen: 9	|	Avg Reward: 0.0	|	Total Reward: 12.8	|
Current Gen: 10	|	Avg Reward: 0.0	|	Total Reward: 3.5	|
Current Gen: 11	|	Avg Reward: 0.0	|	Total Reward: 12.3	|
Current Gen: 12	|	Avg Reward: 0.0	|	Total Reward: 5.8	|
Current Gen: 13	|	Avg Reward: 0.0	|	Total Reward: 3.9	|
Current Gen: 14	|	Avg Reward: 0.0	|	Total Reward: 12.9	|
Current Gen: 15	|	Avg Reward: 0.0	|	Total Reward: 3.7	|
Current Gen: 16	|	Avg Reward: 0.0	|	Total Reward: 32.8	|
Current Gen: 17	|	Avg Reward: 0.0	|	Total Reward: 28.8	|
Current Gen: 18	|	Avg Reward: 0.0	|	Total Rewa